In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation,prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Here're just a preparatory phase for prover
- Create dummy data associated with their dataset for verifier to use for generating onnx model.
- Generate data commitment for their dataset.

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [4]
selected_columns = ['x', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

## Step 2
- User (Verifier) defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `computation_to_model()` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`, while also export `precal_witness_arr.json` which is the constant witness needed for verifier to generate the same onnx.

In [6]:
from zkstats.computation import State, computation_to_model


def computation(state: State, x: list[torch.Tensor]):
    filter = x[1]>3
    filtered_x = torch.where(filter, x[0], 2.0)
    return [torch.sum(filtered_x)]


error = 0.01

_, prover_model = computation_to_model(computation,precal_witness_path, True, error )
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:305: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return x[0][0][0][0]-x[0][0][0][0]+torch.tensor(1.0), result
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where): Y has inconsistent type tensor(double) (Triggered internally at /Users/runner/work/pytorch/p

==== Generate & Calibrate Setting ====
scale:  [4]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":10,"lookup_range":[-60,166],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":2624,"total_assignments":92,"total_const_size":9,"model_instance_shapes":[[1],[1]],"model_output_scales":[4,4],"model_input_scales":[4,4],"module_sizes":{"kzg":[],"poseidon":[2624,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715672081408}


## Step 3
- User (Verifier) gets `precal_witness_arr.json`, and generate the same onnx format with same parameter.

In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

## Step 4
- Verifier sets up vk, pk.
- Prover generates proof with `prover_gen_proof`


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.6420130729675293 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 29.0
==== Generating Proof ====
proof:  {'instances': [['eb0bb15179f97ca814dc819c4913827b8756888b2796ecee8371bec8aa1ed21d', '234d9836ef5dab23b8e75e1048d5dbfbbaa3bc888b6d1387e4b8b8b607e97d08', '1000000000000000000000000000000000000000000000000000000000000000', 'd001000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x00f7b6d89af7f6227b595e8e3ac56754d3c7d964215c6c99c1f740c223d931c11672825f50f35d6209992cdfb37267e01e5ea369d7839815759b0f11732ce8e40ee134e6b2ca81bcb5871c4b71f22cbce16f02b8216114c4b48756ecb87c76b61be755aae3e4259aefe1f57a1e1a2acc871f18fa465917866717a8b25a5ff53b179c14b74c4432d57ed2ff89a9e7bf6a4ab78370d65d1035ad6204043f229648266fc818ae738b05fc5b85a16e62f142fafadc88bdaeecdbe6599f507f1c7b951c9c9ebbbd755b6eb0941f5cbec4e933c8b1db90188ddabb52ee221dcf8cf846169f15b82bdac510afc85ecc3a0f611b2dd037a1afb5d41e24ef09224a0acda218cd2780ae7

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)

[29.0]